<a href="https://colab.research.google.com/github/dangcaptkd/CS114.K21-/blob/master/sarcasm_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lấy data từ kaggle!



In [27]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"quyennguyenthinh","key":"78c6c8efa90a3136ef03e49e71fb5d9b"}'}

In [28]:
!ls -lha kaggle.json
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 72 May  8 02:47 kaggle.json


In [29]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection

  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 54.6MB/s]


In [30]:
!unzip news-headlines-dataset-for-sarcasm-detection

Archive:  news-headlines-dataset-for-sarcasm-detection.zip
  inflating: Sarcasm_Headlines_Dataset.json  
  inflating: Sarcasm_Headlines_Dataset_v2.json  


# new section

In [0]:
import pandas as pd, numpy as np, re, time
from nltk.stem.porter import PorterStemmer

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [0]:
# Loading data from json file
data = pd.read_json('Sarcasm_Headlines_Dataset.json', lines = True)

Xem thử 5 hàng đầu của dataset 

In [35]:
data.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


Check the null value

In [36]:
print(data.isnull().any(axis = 0))

article_link    False
headline        False
is_sarcastic    False
dtype: bool


# clean data!

Ở cột headline sẽ có 1 vài kí tự cần được loại bỏ, dùng regular expression để loại bỏ các kí tự này

In [0]:
data['headline'] = data['headline'].apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))

Gán feature và label 

In [0]:
features = data['headline']
labels = data['is_sarcastic']

Steaming data

In [0]:
ps = PorterStemmer()
features = features.apply(lambda x: x.split())
features = features.apply(lambda x : ' '.join([ps.stem(word) for word in x]))

Vector hóa các tính năng bằng cách sử dụng TF-IDF Vectorizer (chỗ này để làm gì ta??)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features = 5000)
features = list(features)
features = tv.fit_transform(features).toarray()

# Training and testing data

getting training and testing data

In [0]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = .05, random_state = 0)

model 1: Using linear support vector classifier

In [0]:
lsvc = LinearSVC()
# training the model
lsvc.fit(features_train, labels_train)
# getting the score of train and test data
print('train:', lsvc.score(features_train, labels_train)) 
print('test:', lsvc.score(features_test, labels_test))   

train: 0.9093524612777362
test: 0.8375748502994012


model 2: Using Gaussuan Naive Bayes

In [0]:
gnb = GaussianNB()
gnb.fit(features_train, labels_train)
print('train:', gnb.score(features_train, labels_train))  
print('test:', gnb.score(features_test, labels_test))    

train: 0.7886335868836952
test: 0.7380239520958084


model 3: Logistic Regression

In [0]:
lr = LogisticRegression()
lr.fit(features_train, labels_train)
print('train:', lr.score(features_train, labels_train))   
print('test:', lr.score(features_test, labels_test))     

train: 0.8816458440074094
test: 0.8308383233532934


model 4: Random Forest Classifier

In [0]:
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
rfc.fit(features_train, labels_train)
print('train:', rfc.score(features_train, labels_train)) 
print('test:', rfc.score(features_test, labels_test))    

train: 0.9882946439128207
test: 0.7971556886227545


# Some thing new hasn't named yet

In [0]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, LSTM, Bidirectional, Dropout, Conv1D, MaxPool1D
from keras.preprocessing.text import Tokenizer

In [0]:
MAX_SEQ_LENGTH = 40
MAX_WORDS = 20000

In [134]:
data.is_sarcastic.value_counts()

0    14985
1    11724
Name: is_sarcastic, dtype: int64

In [0]:
tok = Tokenizer(num_words = MAX_WORDS) # keeping 10000 now for first iteration
tok.fit_on_texts(data.headline)
seqs = tok.texts_to_sequences(data.headline)

In [0]:
data = pad_sequences(seqs, MAX_SEQ_LENGTH)

In [137]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features = 40)
features = list(features)
features = tv.fit_transform(features).toarray()

AttributeError: ignored

In [127]:
data.shape

(26709, 40)

In [80]:
features.shape

(26709, 40)

In [0]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = .1, random_state = 0)

In [117]:
print('features_train:', features_train.shape)
print('labels_train:', labels_train.shape)
print('features_test:', features_test.shape)
print('labels_test:', labels_test.shape)

features_train: (24038, 40)
labels_train: (24038,)
features_test: (2671, 40)
labels_test: (2671,)


In [101]:
EMB_DIM = 6
def fcmodel():
    model = Sequential()
    model.add(Embedding(input_dim=MAX_WORDS, output_dim= EMB_DIM, input_length=MAX_SEQ_LENGTH))    
    
    # Flatten Layer
    model.add(Flatten())
    
    # FC1
    model.add(Dense(64, activation='relu'))
    
    # Output layer
    model.add(Dense(1, activation='sigmoid'))
    
    # print model summary
    model.summary()
    
    # When using pretrained embeddings
    #model.layers[0].set_weights([embedding_matrix])
    #model.layers[0].trainable = False
              
    # Compile the model
    model.compile(optimizer = 'rmsprop',
                 loss = 'binary_crossentropy',
                 metrics = ['acc'])
    return model

fcmod = fcmodel()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 40, 6)             120000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 240)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                15424     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 135,489
Trainable params: 135,489
Non-trainable params: 0
_________________________________________________________________


In [0]:
EPOCHS = 9
BATCH_SIZE =512

In [115]:
fchist = fcmod.fit(features_train, labels_train,
                   epochs = EPOCHS,
                   batch_size = BATCH_SIZE,
                   validation_data = (features_test, labels_test))

Train on 24038 samples, validate on 2671 samples
Epoch 1/9
24038/24038 [==============================] - 0s 10us/step - loss: 0.6643 - acc: 0.5955 - val_loss: 0.6640 - val_acc: 0.5915
Epoch 2/9
24038/24038 [==============================] - 0s 9us/step - loss: 0.6640 - acc: 0.5957 - val_loss: 0.6641 - val_acc: 0.5915
Epoch 3/9
24038/24038 [==============================] - 0s 9us/step - loss: 0.6640 - acc: 0.5951 - val_loss: 0.6636 - val_acc: 0.5923
Epoch 4/9
24038/24038 [==============================] - 0s 9us/step - loss: 0.6638 - acc: 0.5955 - val_loss: 0.6635 - val_acc: 0.5923
Epoch 5/9
24038/24038 [==============================] - 0s 9us/step - loss: 0.6639 - acc: 0.5954 - val_loss: 0.6637 - val_acc: 0.5923
Epoch 6/9
24038/24038 [==============================] - 0s 9us/step - loss: 0.6639 - acc: 0.5955 - val_loss: 0.6636 - val_acc: 0.5923
Epoch 7/9
24038/24038 [==============================] - 0s 10us/step - loss: 0.6637 - acc: 0.5954 - val_loss: 0.6639 - val_acc: 0.5923
Epoc

In [116]:
fchist.history['acc']

[0.5954738,
 0.59568185,
 0.59505785,
 0.59551543,
 0.5954322,
 0.5954738,
 0.5953906,
 0.5954322,
 0.59551543]